In [1]:
import torch
import torch.nn as nn
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error

In [2]:

train_path = "../data/split/train/train.csv"
test_path = "../data/split/test/test.csv"

df_train = pd.read_csv(train_path)
df_test = pd.read_csv(test_path)

feature_cols = ['area', 'bedrooms', 'bathrooms']
target_col = 'price'


In [3]:

scaler_area = StandardScaler()
scaler_price = StandardScaler()

df_train['area'] = scaler_area.fit_transform(df_train[['area']])
df_test['area']  = scaler_area.transform(df_test[['area']])

df_train['price'] = scaler_price.fit_transform(df_train[['price']])
df_test['price']  = scaler_price.transform(df_test[['price']])

In [4]:
X_train = torch.tensor(df_train[feature_cols].values, dtype=torch.float32)
y_train = torch.tensor(df_train[[target_col]].values, dtype=torch.float32)

X_test  = torch.tensor(df_test[feature_cols].values, dtype=torch.float32)
y_test  = torch.tensor(df_test[[target_col]].values, dtype=torch.float32)


In [5]:
model = nn.Linear(in_features=X_train.shape[1], out_features=1)

criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [9]:

epochs = 8000

for epoch in range(epochs):
    y_pred = model(X_train)
    loss = criterion(y_pred, y_train)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 500 == 0:
        print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.6f}")

Epoch 500/8000, Loss: 0.891863
Epoch 1000/8000, Loss: 0.891700
Epoch 1500/8000, Loss: 0.891649
Epoch 2000/8000, Loss: 0.891633
Epoch 2500/8000, Loss: 0.891628
Epoch 3000/8000, Loss: 0.891626
Epoch 3500/8000, Loss: 0.891625
Epoch 4000/8000, Loss: 0.891625
Epoch 4500/8000, Loss: 0.891625
Epoch 5000/8000, Loss: 0.891625
Epoch 5500/8000, Loss: 0.891625
Epoch 6000/8000, Loss: 0.891625
Epoch 6500/8000, Loss: 0.891625
Epoch 7000/8000, Loss: 0.891625
Epoch 7500/8000, Loss: 0.891625
Epoch 8000/8000, Loss: 0.891625


In [7]:
print("\n=== MODEL PARAMETERS ===")
print("Weights:", model.weight.detach().numpy())
print("Bias:", model.bias.detach().numpy())



=== MODEL PARAMETERS ===
Weights: [[0.00480188 0.04674863 0.0430467 ]]
Bias: [-0.22593157]


In [8]:
y_pred = model(X_test).detach().numpy()

print("\n=== EVALUATION ===")
print("R²:", r2_score(y_test.numpy(), y_pred))
print("RMSE:", mean_squared_error(y_test.numpy(), y_pred) ** 0.5)


=== EVALUATION ===
R²: 0.059758663177490234
RMSE: 1.0921577307664794
